In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint
import urllib.parse

# Import API key
from my_api_config import g_key
from my_api_config import api_key

In [20]:
geo_test_pd = pd.read_csv('../Resources/geo_test100.csv', encoding='utf-8')
del geo_test_pd['Unnamed: 0']
del geo_test_pd['Unnamed: 0.1']
geo_test_pd

,country,description,designation,points,price,province,region_1,region_2,title,variety,winery
0,US,This inexpensive white wine does a good job of...,NaN,85,10.0,California,Central Coast,Central Coast,Tamás 2008 Pinot Grigio (Central Coast),Pinot Grigio,Tamás
1,US,The fruit intensity and depth in this full-bod...,Single Vineyard,93,72.0,California,Anderson Valley,NaN,Donum 2013 Single Vineyard Pinot Noir (Anderso...,Pinot Noir,Donum
2,Italy,This is a sharply focused expression of the Bu...,Bussia,91,52.0,Piedmont,Barolo,NaN,Giacosa Fratelli 2004 Bussia (Barolo),Nebbiolo,Giacosa Fratelli
3,US,Incredibly rich blackberry-jam aromas meet wit...,NaN,93,42.0,California,Paso Robles Willow Creek District,Central Coast,Dunning Vineyards 2013 Cabernet Sauvignon (Pas...,Cabernet Sauvignon,Dunning Vineyards
4,US,The expected aromas of cut grass and cat pee g...,NaN,87,15.0,California,Edna Valley,Central Coast,Edna Valley Vineyard 2013 Sauvignon Blanc (Edn...,Sauvignon Blanc,Edna Valley Vineyard
...,...,...,...,...,...,...,...,...,...,...,...
95,Italy,This barrique-aged Chardonnay opens with delic...,Benefizio Riserva,89,45.0,Tuscany,Pomino,NaN,Marchesi de' Frescobaldi 2012 Benefizio Riserv...,Chardonnay,Marchesi de' Frescobaldi
96,Spain,"Savory and deep given the price, with dark col...",Riscal 1860,86,9.0,Northern Spain,Vino de la Tierra de Castilla y León,NaN,Riscal 2006 Riscal 1860 Tempranillo (Vino de l...,Tempranillo,Riscal
97,Italy,This is aged for two years in French oak to sh...,NaN,89,32.0,Piedmont,Barbaresco,NaN,Bosio 2007 Barbaresco,Nebbiolo,Bosio
98,Canada,"Tannic and astringent at first taste, this win...",Qwam Qwmt,89,35.0,British Columbia,Okanagan Valley,NaN,Nk'Mip Cellars 2009 Qwam Qwmt Syrah (Okanagan ...,Syrah,Nk'Mip Cellars


In [ ]:
geo_test_pd["winery"][0], geo_test_pd["province"][0],geo_test_pd["description"][0]

In [ ]:
# Set parameters to search for locations.

query = f'{geo_test_pd["winery"][0]},winery,{geo_test_pd["region_1"][0]},{geo_test_pd["region_2"][0]}'
place = urllib.parse.quote(query)
province = urllib.parse.quote(geo_test_pd["province"][0])

# base url
# base_url = (f'https://api.opencagedata.com/geocode/v1/json?q={place}&key={api_key}')
base_url = (f'https://api.opencagedata.com/geocode/v1/json?q={place}%2C%20{province}&key={api_key}')
# run a request using our params dictionary
response = requests.get(base_url).json()
pprint.pprint(response)

In [ ]:
base_url

In [ ]:
province

In [ ]:
response['results'][0]['confidence']

In [ ]:
len(response['results'])

In [ ]:
# Selecting responses with medium to high confidence score
num_responses = len(response['results'])

for x in range(num_responses):
#     print(x)
    confidence = response['results'][x]['confidence'] 
    if confidence >=5:
        print(confidence)
    else:
        print(f'not confident: {confidence}')

# len(response['results'])

In [3]:
# Hit the OpenCageData API for each winery.
# Set up a Dictionary to hold reponse info for:
town=[]
lat=[]
lng=[]

print("Beginning search for nearby wineries... \n-----------------------------")

# Loop through the geo_test_pd and get a lat/long for each winery
for index, row in geo_test_pd.iterrows():
   
    query = f'{row["winery"]},{row["region_1"]},{row["province"]},{row["country"]}'
    place = urllib.parse.quote(query)
#     province = urllib.parse.quote(row["province"])
#     country = row["country"]

    base_url = (f'https://api.opencagedata.com/geocode/v1/json?q={place}&key={api_key}')

    try:
        print(f"Searching for winery {query}")
        response = requests.get(base_url).json()
#         print(place)
#         print(province)
        town.append(response['results'][0]['components']['town'])
        lat.append(response['results'][0]['geometry']['lat'])
        lng.append(response['results'][0]['geometry']['lng'])
        print(f'---> {query} appended')
        print(" ")
    except:
        print(f"X--X No wineries found. Skipping {query}...")
        print(" ")
        town.append(None)
        lat.append(None)
        lng.append(None)

Beginning search for nearby wineries... 
-----------------------------
Searching for winery Tamás,Central Coast,California,US
---> Tamás,Central Coast,California,US appended
 
Searching for winery Donum,Anderson Valley,California,US
---> Donum,Anderson Valley,California,US appended
 
Searching for winery Giacosa Fratelli,Barolo,Piedmont,Italy
---> Giacosa Fratelli,Barolo,Piedmont,Italy appended
 
Searching for winery Dunning Vineyards,Paso Robles Willow Creek District,California,US
---> Dunning Vineyards,Paso Robles Willow Creek District,California,US appended
 
Searching for winery Edna Valley Vineyard,Edna Valley,California,US
X--X No wineries found. Skipping Edna Valley Vineyard,Edna Valley,California,US...
 
Searching for winery Château Sainte Marie,Bordeaux Supérieur,Bordeaux,France
---> Château Sainte Marie,Bordeaux Supérieur,Bordeaux,France appended
 
Searching for winery Chateau Ste. Michelle,Horse Heaven Hills,Washington,US
X--X No wineries found. Skipping Chateau Ste. Michell

---> Novy,Santa Lucia Highlands,California,US appended
 
Searching for winery Château de Seguin,Bordeaux,Bordeaux,France
X--X No wineries found. Skipping Château de Seguin,Bordeaux,Bordeaux,France...
 
Searching for winery Bouchard Père & Fils,Corton,Burgundy,France
X--X No wineries found. Skipping Bouchard Père & Fils,Corton,Burgundy,France...
 
Searching for winery Gustave Lorentz,Crémant d'Alsace,Alsace,France
X--X No wineries found. Skipping Gustave Lorentz,Crémant d'Alsace,Alsace,France...
 
Searching for winery El Grifo,Lanzarote,Spanish Islands,Spain
X--X No wineries found. Skipping El Grifo,Lanzarote,Spanish Islands,Spain...
 
Searching for winery Blackbird Vineyards,Napa Valley,California,US
---> Blackbird Vineyards,Napa Valley,California,US appended
 
Searching for winery Langtry,Guenoc Valley,California,US
---> Langtry,Guenoc Valley,California,US appended
 
Searching for winery Shalestone,Finger Lakes,New York,US
---> Shalestone,Finger Lakes,New York,US appended
 
Searching 

In [32]:
# Hit the OpenCageData API for each winery.
# Set up a Dictionary to hold reponse info for:
town=[]
lat=[]
lng=[]


print("Beginning search for nearby wineries... \n-----------------------------")

# Loop through the geo_test_pd and get a lat/long for each winery
for index, row in geo_test_pd.iterrows():
   
    query = f'winery,{row["region_1"]},{row["province"]},{row["country"]}'
    place = urllib.parse.quote(query)
#     province = urllib.parse.quote(row["province"])
#     country = row["country"]

    base_url = (f'https://api.opencagedata.com/geocode/v1/json?q={place}&key={api_key}')

    try:
        print(f"Searching for winery: {query}")
        response = requests.get(base_url).json()
#         print(place)
#         print(province)
        
        # Selecting responses with medium to high confidence score
        num_responses = len(response['results'])
        print(f'num of responses {num_responses}')
        for x in range(num_responses):
            print(f'response{x}')
            confidence = response['results'][x]['confidence'] 
            if confidence >4:
                print(f'confidence:{confidence}')
                town.append(response['results'][x]['components']['town'])
                lat.append(response['results'][x]['geometry']['lat'])
                lng.append(response['results'][x]['geometry']['lng'])
                print(f'---> {query} appended')
                print(" ")
                break
            elif x == (num_responses - 1):
                print(f'Not confident: {confidence} - Skipping {query}...')
                print(" ")
                town.append(None)
                lat.append(None)
                lng.append(None)
            else:
                print(f'Not confident: {confidence} - Next result...')
                print(" ")
    except:
        print(f"X--X No wineries found. Skipping {query}...")
        print(" ")
        town.append(None)
        lat.append(None)
        lng.append(None)

Beginning search for nearby wineries... 
-----------------------------
Searching for winery: winery,Central Coast,California,US
num of responses 2
response0
Not confident: 4 - Next result...
 
response1
Not confident: 1 - Skipping winery,Central Coast,California,US...
 
Searching for winery: winery,Anderson Valley,California,US
num of responses 2
response0
Not confident: 4 - Next result...
 
response1
confidence:9
X--X No wineries found. Skipping winery,Anderson Valley,California,US...
 
Searching for winery: winery,Barolo,Piedmont,Italy
num of responses 1
response0
confidence:9
---> winery,Barolo,Piedmont,Italy appended
 
Searching for winery: winery,Paso Robles Willow Creek District,California,US
num of responses 2
response0
confidence:5
---> winery,Paso Robles Willow Creek District,California,US appended
 
Searching for winery: winery,Edna Valley,California,US
num of responses 8
response0
confidence:9
X--X No wineries found. Skipping winery,Edna Valley,California,US...
 
Searching f

num of responses 10
response0
confidence:9
---> winery,Volnay,Burgundy,France appended
 
Searching for winery: winery,Sta. Rita Hills,California,US
num of responses 2
response0
Not confident: 3 - Next result...
 
response1
Not confident: 1 - Skipping winery,Sta. Rita Hills,California,US...
 
Searching for winery: winery,Veronese,Veneto,Italy
num of responses 1
response0
confidence:9
---> winery,Veronese,Veneto,Italy appended
 
Searching for winery: winery,Alsace,Alsace,France
num of responses 2
response0
Not confident: 1 - Next result...
 
response1
Not confident: 3 - Skipping winery,Alsace,Alsace,France...
 
Searching for winery: winery,Columbia Valley (WA),Washington,US
num of responses 2
response0
Not confident: 3 - Next result...
 
response1
confidence:9
X--X No wineries found. Skipping winery,Columbia Valley (WA),Washington,US...
 
Searching for winery: winery,Monterey County,California,US
num of responses 5
response0
confidence:9
X--X No wineries found. Skipping winery,Monterey C

num of responses 10
response0
confidence:9
---> winery,Paso Robles,California,US appended
 
Searching for winery: winery,Amarone della Valpolicella Classico,Veneto,Italy
num of responses 1
response0
Not confident: 1 - Skipping winery,Amarone della Valpolicella Classico,Veneto,Italy...
 
Searching for winery: winery,Columbia Valley (WA),Washington,US
num of responses 2
response0
Not confident: 3 - Next result...
 
response1
confidence:9
X--X No wineries found. Skipping winery,Columbia Valley (WA),Washington,US...
 
Searching for winery: winery,Alexander Valley,California,US
num of responses 2
response0
Not confident: 3 - Next result...
 
response1
confidence:7
X--X No wineries found. Skipping winery,Alexander Valley,California,US...
 
Searching for winery: winery,Columbia Valley (WA),Washington,US
num of responses 2
response0
Not confident: 3 - Next result...
 
response1
confidence:9
X--X No wineries found. Skipping winery,Columbia Valley (WA),Washington,US...
 
Searching for winery: wi

In [33]:
town, lat, lng
len(town)

100

In [34]:
# Store the highest confidence result into the DataFrame.
geo_test_pd['town'] = town
geo_test_pd['lat'] = lat
geo_test_pd['lng'] = lng
geo_test_pd2 = geo_test_pd.dropna(subset=['lat', 'lng'])
geo_test_pd2

,country,description,designation,points,price,province,region_1,region_2,title,variety,winery,town,lat,lng
2,Italy,This is a sharply focused expression of the Bu...,Bussia,91,52.0,Piedmont,Barolo,NaN,Giacosa Fratelli 2004 Bussia (Barolo),Nebbiolo,Giacosa Fratelli,Barolo,44.610940,7.942840
3,US,Incredibly rich blackberry-jam aromas meet wit...,NaN,93,42.0,California,Paso Robles Willow Creek District,Central Coast,Dunning Vineyards 2013 Cabernet Sauvignon (Pas...,Cabernet Sauvignon,Dunning Vineyards,Santa Margarita,35.303644,-120.373697
5,France,"Ripe and toasty, this is a rich, full-bodied a...",NaN,90,15.0,Bordeaux,Bordeaux Supérieur,NaN,Château Sainte Marie 2011 Bordeaux Supérieur,Bordeaux-style Red Blend,Château Sainte Marie,Bordeaux,44.840440,-0.580500
7,US,"Lush black plum, bramble and black pepper ming...",Estate,87,34.0,Virginia,Middleburg,NaN,Casanel 2014 Estate Cabernet Sauvignon (Middle...,Cabernet Sauvignon,Casanel,Middleburg,38.973562,-77.706093
11,Italy,A steady protagonist in the greater Brunello f...,NaN,89,70.0,Tuscany,Brunello di Montalcino,NaN,PianCornello 2008 Brunello di Montalcino,Sangiovese Grosso,PianCornello,Montalcino,43.053970,11.488530
16,US,"A light, bright, affordable quaffer, this yout...",Rosé of,90,22.0,California,Sonoma Coast,Sonoma,Brick & Mortar 2015 Rosé of Pinot Noir (Sonoma...,Pinot Noir,Brick & Mortar,Sonoma,38.292088,-122.433447
20,US,This Syrah-Zin blend is forward in jammy raspb...,Le Pique-Nique,84,14.0,California,Sonoma Valley,Sonoma,Wellington 2007 Le Pique-Nique Rosé (Sonoma Va...,Rosé,Wellington,Sonoma,38.292088,-122.433447
29,France,"Structured and firm, with dense, dry tannins, ...",NaN,88,18.0,Bordeaux,Cadillac Côtes de Bordeaux,NaN,Château du Cros 2015 Cadillac Côtes de Bordeaux,Bordeaux-style Red Blend,Château du Cros,Bordeaux,44.840440,-0.580500
32,US,"Abundant dark fruit flavors, particularly Bing...",NaN,91,39.0,California,Red Hills Lake County,NaN,Sol Rouge 2013 Cabernet Sauvignon (Red Hills L...,Cabernet Sauvignon,Sol Rouge,California City,35.130657,-117.937412
39,France,The small 11-acre estate is situated at the no...,NaN,92,35.0,Bordeaux,Margaux,NaN,Château Bellevue de Tayac 2014 Margaux,Bordeaux-style Red Blend,Château Bellevue de Tayac,Bordeaux,44.840440,-0.580500


In [35]:
len(geo_test_pd2)


30

In [11]:
geo_test_pd2.to_csv('../Resources/geo_test2.csv', encoding='utf-8')

In [ ]:
## If closing Jupyter Notebook, read in the saved geo_test2.csv instead of re-reunning OpenCageData API calls
geo_test_pd2 = pd.read_csv('../Resources/geo_test2.csv', encoding='utf-8')

In [36]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [37]:
# Store 'lat' and 'lng' into  locations 
locations = geo_test_pd2[["lat", "lng"]].astype(float)
# locations
rating = geo_test_pd2["points"].astype(float)
price = geo_test_pd2["price"].astype(float)

# Create Humidity Heatmap layer
fig = gmaps.figure()

heat_rating = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=True, max_intensity=rating.max())
heat_price = gmaps.heatmap_layer(locations, weights=price, 
                                 dissipating=True, max_intensity=rating.max(), opacity=0.3, gradient=['white','blue'])

fig.add_layer(heat_rating)
fig.add_layer(heat_price)

fig

Figure(layout=FigureLayout(height='420px'))

In [38]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{winery}</dd>
<dt>Vintage</dt><dd>{title}</dd>
<dt>Rating</dt><dd>{points}</dd>
<dt>Price</dt><dd>{price}</dd>
<dt>Province</dt><dd>{province}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
winery_info = [info_box_template.format(**row) for index, row in geo_test_pd2.iterrows()]
locations = geo_test_pd2[["lat", "lng"]] 


In [39]:
# Add marker layer ontop of heat map
fig = gmaps.figure()

# rating = geo_test_pd2['points']

# def color(rating): 
#     if rating in range(95,100): 
#         col = 'blue'
#     elif rating in range(90,94): 
#         col = 'green'
#     elif rating in range(80,89): 
#         col = 'orange'
#     else: 
#         col='red'
#     return col 

marker_layer = gmaps.marker_layer(locations, info_box_content=winery_info, display_info_box=None)
# marker_layer.markers[0].label = 'C'  # markers[0].label changes the center of the red Google Marker

# wine_rating_layer = gmaps.symbol_layer(locations, fill_color=color(rating),stroke_color="black", scale=5)


# fid.add_layer(wine_rating_layer)
fig.add_layer(heat_rating)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

# Other APIs tried

In [ ]:
# # Set parameters to search for locations within 5000 meters.
# # geocoordinates
# target_coordinates = "34.01,-118.81"
# target_search = "wine"
# target_radius = 5000
# target_type = "winery"

# # set up a parameters dictionary
# params = {
#     "location": target_coordinates,
#     "keyword": target_search,
#     "radius": target_radius,
#     "type": target_type,
#     "key": g_key}

# # base url
# base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# # run a request using our params dictionary
# response = requests.get(base_url, params=params).json()
# pprint.pprint(response)

In [ ]:
# # Set parameters to Text Search search for winery locations.

# target_search = "Rainstorm+winery"
# target_type = "winery"

# # set up a parameters dictionary
# params = {
#     "query": target_search,
#     "type": target_type,
#     "key": g_key}

# # base url
# base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"

# # run a request using our params dictionary
# response = requests.get(base_url, params=params).json()
# pprint.pprint(response)

In [ ]:
# # Set parameters Find Place From Text to search for winery locations.

# target_search = "Rainstorm"
# target_type = "winery,wine"

# # set up a parameters dictionary
# params = {
#     "input": target_search,
#     "inputtype": "textquery",
#     "type": target_type,
#     "key": g_key}

# # base url
# base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"

# # run a request using our params dictionary
# response = requests.get(base_url, params=params).json()
# pprint.pprint(response)